##  A 5-Hidden Layer Architecture
What is happening here?
    [1] A traditional ANN model.
    [2] Import and preprocessing data.
    [3] Trigonometric transformations, which converting the original angle plies into sine and cosine values range between -1 and 1 to helps normalize data.
    [4] Create a neural network model with specific hidden layer and the model uses the Sequential API from Keras, starting with an input layer, followed by a dense (fully connected) hidden layer with ReLU activation, and then an output layer with linear activation (for regression tasks).
    [5] Model Training and Evaluation: run over a range of different number of neurons (from 20 to 80 with step of 20) to tune the model's hyperparameters.
    For each number of neurons: A KerasRegressor wrapper is instantiated with the defined neural network model and training settings (epochs, batch size).
    5-Fold Cross-Validation for robust evaluates the model's performance metrics (MSE, MAE, R-squared) on each fold of the data and to avoid overfitting.
    [6] Model Evaluation (mean and standard deviation of performance metrics).
    [7] Plot Mean of(R2, MAE, RMSE) vs number of neurons.

In [3]:
import pandas as pd 
import numpy as np                                             
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from sklearn.base import TransformerMixin
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import KFold, cross_validate
import matplotlib.patches as mpatches

## Import data
bucklingdata = pd.read_csv("CBL_SimulationResults.csv", encoding='cp1252')

## Convert the hole_column from integer into a floating data type
bucklingdata['Number of holes, nh'] = bucklingdata['Number of holes, nh'].astype('float64')
bucklingdata['Web opening diameter, Do (mm)'] = bucklingdata['Web opening diameter, Do (mm)'].astype('float64')

## Trigonometric transformation of the ply angles: We develop a trigonometric transformation function to convert the angles (this is important)
# Create a transformation class here for the angle plies:
class TrigonometricTransformer(TransformerMixin):
    def __init__(self, start_column=1, end_column=-1):
        self.start_column = start_column
        self.end_column = end_column
        self.transform_functions = [np.sin, np.cos]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.copy()
        columns_to_transform = X.columns[self.start_column:self.end_column]

        for col in columns_to_transform:
            for func in self.transform_functions:
                X_transformed[col + '_' + func.__name__] = func(X[col])

        # Remove the original columns
        X_transformed.drop(columns=columns_to_transform, inplace=True)

        return X_transformed

# Specify the input and output datasets
input_data = bucklingdata.drop(['Critical Buckling Load (N)','Critical Buckling Load (kN)'], axis=1)
output_data = bucklingdata['Critical Buckling Load (kN)']

# Apply the transformation on the ply columns
transformer = TrigonometricTransformer(start_column=3, end_column=11)
# Transform the input DataFrame
transformed_bucklingdata = transformer.fit_transform(input_data)

# The input dimension for the Neural Network is then obtained as. Then, the input_size will use in following: model.add(Input(shape=(19,)))
input_size = transformed_bucklingdata.shape[1]
#print(input_size)

## Define a specific neural network model with five hidden layer
def create_model5H(neurons_list):
    model = Sequential()
    model.add(Input(shape=(19,)))
    hp_activation = 'selu'
    model.add(Dense(neurons_list[0], activation=hp_activation))
    model.add(Dense(neurons_list[1], activation=hp_activation))
    model.add(Dense(neurons_list[2], activation=hp_activation))
    model.add(Dense(neurons_list[3], activation=hp_activation))
    model.add(Dense(neurons_list[4], activation=hp_activation))
    model.add(Dense(1, activation='linear'))
    hp_learning_rate = 0.01
    model.compile(loss='mse',
                  optimizer=Adam(learning_rate=hp_learning_rate),
                  metrics=['mae'])
    return model

# Pass the input and output dataframes to X , y (this is just convention in Python ML)
X = transformed_bucklingdata.copy()
y = output_data.copy()

# Specify the range for neurons in the first and second hidden layers
min_neurons = 20
max_neurons = 80
step = 20

neuron_configs = []


# Generate configurations with increasing neurons in all three layers
for neurons1 in range(min_neurons, max_neurons + 1, step):
    for neurons2 in range(min_neurons, max_neurons + 1, step):
        for neurons3 in range(min_neurons, max_neurons + 1, step):
            for neurons4 in range(min_neurons, max_neurons + 1, step):
                for neurons5 in range(min_neurons, max_neurons + 1, step):
                    neuron_configs.append((neurons1, neurons2, neurons3, neurons4, neurons5))

# Define lists to store results
results = []


for i, (p1, p2, p3, p4, p5) in enumerate(neuron_configs):
    print(f"Training model with configuration {i+1}: Hidden Layer 1={p1}, Hidden Layer 2={p2}, Hidden Layer 3={p3}, Hidden Layer 4={p4}, Hidden Layer 5={p5}")
    
    keras_regressor = KerasRegressor(model=lambda: create_model5H([neurons1, neurons2, neurons3, neurons4, neurons5]), epochs=150, batch_size=32, verbose=0)
    
    # Use cross_val_score to evaluate the model
    # Cross validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = cross_validate(keras_regressor, X, y, cv=kf,
                             scoring=('neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'))
    mse_scores = -cv_results['test_neg_mean_squared_error']
    mae_scores = -cv_results['test_neg_mean_absolute_error']
    r2_scores = cv_results['test_r2']                           # R-squared scores are already positive
         
    # Store the evaluation metrics in a DataFrame
    result_df = pd.DataFrame({
        'Configuration': i + 1,
        'Neurons_1HL': p1,
        'Neurons_2HL': p2,
        'Neurons_3HL': p3,
        'Neurons_4HL': p4,
        'Neurons_5HL': p5,
        'Mean RMSE': np.sqrt(mse_scores).mean(),
        'Std RMSE': np.sqrt(mse_scores).std(),
        'Mean MAE': mae_scores.mean(),
        'Std MAE': mae_scores.std(),
        'Mean R-squared': r2_scores.mean(),
        'Std R-squared': r2_scores.std()
    }, index=[0])
    
    results.append(result_df)

# Concatenate all results into a single DataFrame
results_df = pd.concat(results, ignore_index=True)

# Save results to an Excel file
results_df.to_excel('5HL_Neurons_Network_Results_KFold_CV.xlsx', index=False)
print()
print("------------Congrats! You have successfully save the results to 5HL_Neurons_Network_Results_KFold_CV------------")

Training model with configuration 1: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=20, Hidden Layer 5=20
Training model with configuration 2: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=20, Hidden Layer 5=40
Training model with configuration 3: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=20, Hidden Layer 5=60
Training model with configuration 4: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=20, Hidden Layer 5=80
Training model with configuration 5: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=40, Hidden Layer 5=20
Training model with configuration 6: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=40, Hidden Layer 5=40
Training model with configuration 7: Hidden Layer 1=20, Hidden Layer 2=20, Hidden Layer 3=20, Hidden Layer 4=40, Hidden Layer 5=60
Training model with configuration 8: Hidden Layer 1=20, Hidden Layer 2=20, Hidden L

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\wongy\PycharmProjects\ML_Buckling_ANN_DNN\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wongy\PycharmProjects\ML_Buckling_ANN_DNN\.venv\Lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\wongy\PycharmProjects\ML_Buckling_ANN_DNN\.venv\Lib\site-packages\scikeras\wrappers.py", line 938, in _fit
    self._fit_keras_model(
  File "C:\Users\wongy\PycharmProjects\ML_Buckling_ANN_DNN\.venv\Lib\site-packages\scikeras\wrappers.py", line 535, in _fit_keras_model
    hist = self.model_

## Plotting Performance Metrics Evaluation Graphs

In [ ]:
# Plot Mean R-squared against neurons
plt.figure(figsize =(10, 10))
width = 0.8
positions_1hl = np.arange(min_neurons, max_neurons + 1, step) - width/1.5
box1 = plt.boxplot([results_df['Mean R-squared'].loc[results_df['Neurons_1HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_1hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='blue', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_2hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box2 = plt.boxplot([results_df['Mean R-squared'].loc[results_df['Neurons_2HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_2hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='green', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_3hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box3 = plt.boxplot([results_df['Mean R-squared'].loc[results_df['Neurons_3HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_3hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='red', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_4hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box4 = plt.boxplot([results_df['Mean R-squared'].loc[results_df['Neurons_4HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_4hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='yellow', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_5hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box5 = plt.boxplot([results_df['Mean R-squared'].loc[results_df['Neurons_5HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_5hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='purple', facecolor='white'),
                   whiskerprops=dict(color='black'))
plt.xticks(np.arange(min_neurons, max_neurons + 1, step), [f'{neurons} Neurons' for neurons in range(min_neurons, max_neurons + 1, step)])
plt.ylabel('Mean R-squared')
plt.title('Mean R-squared vs. Number of Neurons', fontsize = 10)
legend_handles = [
    mpatches.Patch(facecolor='white', edgecolor='blue', linewidth=1, label='1st Hidden Layer'),
    mpatches.Patch(facecolor='white', edgecolor='green', linewidth=1, label='2nd Hidden Layer')
]
plt.legend(handles=legend_handles, loc='upper center')
plt.subplots_adjust(bottom=0.2)
plt.grid(True)
plt.show()



# Plot Mean MAE against neurons
plt.figure(figsize =(10, 10)) 
width = 0.8
positions_1hl = np.arange(min_neurons, max_neurons + 1, step) - width/1.5
box1 = plt.boxplot([results_df['Mean MAE'].loc[results_df['Neurons_1HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_1hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='blue', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_2hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box2 = plt.boxplot([results_df['Mean MAE'].loc[results_df['Neurons_2HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_2hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='green', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_3hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box3 = plt.boxplot([results_df['Mean MAE'].loc[results_df['Neurons_3HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_3hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='red', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_4hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box4 = plt.boxplot([results_df['Mean MAE'].loc[results_df['Neurons_4HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_4hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='yellow', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_5hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box5 = plt.boxplot([results_df['Mean MAE'].loc[results_df['Neurons_5HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_5hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='purple', facecolor='white'),
                   whiskerprops=dict(color='black'))
plt.xticks(np.arange(min_neurons, max_neurons + 1, step), [f'{neurons} Neurons' for neurons in range(min_neurons, max_neurons + 1, step)])
plt.ylabel('Mean MAE')
plt.title('Mean MAE vs. Number of Neurons', fontsize = 10)
legend_handles = [
    mpatches.Patch(facecolor='white', edgecolor='blue', linewidth=1, label='1st Hidden Layer'),
    mpatches.Patch(facecolor='white', edgecolor='green', linewidth=1, label='2nd Hidden Layer')
]
plt.legend(handles=legend_handles, loc='upper center')
plt.subplots_adjust(bottom=0.2)
plt.grid(True)
plt.show()



# Plot Mean RMSE against neurons
plt.figure(figsize =(10, 10))
width = 0.8
positions_1hl = np.arange(min_neurons, max_neurons + 1, step) - width/1.5
box1 = plt.boxplot([results_df['Mean RMSE'].loc[results_df['Neurons_1HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_1hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='blue', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_2hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box2 = plt.boxplot([results_df['Mean RMSE'].loc[results_df['Neurons_2HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_2hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='green', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_3hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box3 = plt.boxplot([results_df['Mean RMSE'].loc[results_df['Neurons_3HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_3hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='red', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_4hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box4 = plt.boxplot([results_df['Mean RMSE'].loc[results_df['Neurons_4HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_4hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='yellow', facecolor='white'),
                   whiskerprops=dict(color='black'))
positions_5hl = np.arange(min_neurons, max_neurons + 1, step) + width/1.5
box5 = plt.boxplot([results_df['Mean RMSE'].loc[results_df['Neurons_5HL'] == neurons] for neurons in range(min_neurons, max_neurons + 1, step)],
                   positions=positions_5hl,
                   widths=width,
                   patch_artist=True,
                   boxprops=dict(color='purple', facecolor='white'),
                   whiskerprops=dict(color='black'))
plt.xticks(np.arange(min_neurons, max_neurons + 1, step), [f'{neurons} Neurons' for neurons in range(min_neurons, max_neurons + 1, step)])
plt.ylabel('Mean RMSE')
plt.title('Mean RMSE vs. Number of Neurons', fontsize = 10)
legend_handles = [
    mpatches.Patch(facecolor='white', edgecolor='blue', linewidth=1, label='1st Hidden Layer'),
    mpatches.Patch(facecolor='white', edgecolor='green', linewidth=1, label='2nd Hidden Layer')
]
plt.legend(handles=legend_handles, loc='upper center')
plt.subplots_adjust(bottom=0.2)
plt.grid(True)
plt.show()